AI BootCamp Project - Zaka AI 

Presented by:
    
    Rewa Rammal
    Elias-Charbel Salameh

"LifeLike Bot Builders"

A Convolutional Neural Network model implementation using the Kaggle Animals-10 dataset having 10 classes:

cat, dog, squirrel, spider, butterfly, horse, cow, sheep, elephant, chicken

Setup: Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.applications import VGG16, ResNet50
import seaborn as sns
from PIL import Image
import pandas as pd
import os
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

train_dir = r'C:\Users\elias\OneDrive\Bureau\Internships\Zaka ai\BootCamp\Project\split_dataset\train'
val_dir = r'C:\Users\elias\OneDrive\Bureau\Internships\Zaka ai\BootCamp\Project\split_dataset\val'
data_dir = r'C:\Users\elias\OneDrive\Bureau\Internships\Zaka ai\BootCamp\Project\dataset'
test_dir = r'C:\Users\elias\OneDrive\Bureau\Internships\Zaka ai\BootCamp\Project\test_set'
        
print("Setup Done")

Functions to be used:

In [ ]:
def display_images_with_labels(generator, model, num_images, rows, columns):
        # Get the class labels
        class_labels = list(generator.class_indices.keys())

        # Get a batch of images and their true labels from the generator
        images, true_labels = next(generator)
        
        # Get predictions for the images
        predictions = model.predict(images)
        predicted_labels = [class_labels[np.argmax(pred)] for pred in predictions]
        
        # Calculate accuracy
        true_indices = np.argmax(true_labels, axis=1)
        accuracy = np.mean(np.equal(true_indices, np.argmax(predictions, axis=1)))

        # Randomly select num_images indices
        indices = random.sample(range(len(images)), num_images)
        
        # Display the images and labels
        fig, axes = plt.subplots(rows, columns, figsize=(15, 15))
        
        for i, ax in enumerate(axes.flat):
                ax.imshow(images[indices[i]])
                true_label = class_labels[np.argmax(true_labels[indices[i]])]
                predicted_label = predicted_labels[indices[i]]
                
                # Check if predicted label matches true label
                if true_label == predicted_label:
                        title_color = 'green'
                else:
                        title_color = 'red'
                
                ax.set_title(f'True Label: {true_label}\nPredicted Label: {predicted_label}', color=title_color)
                ax.axis('off')
        
        plt.tight_layout()
        plt.show()

        print("The accuracy on these 25 images is:", accuracy)

# Function to generate classification report
def generate_classification_report(model, generator, class_labels):
        predictions = model.predict(generator)
        predicted_classes = np.argmax(predictions, axis=1)
        true_classes = generator.classes
        report = classification_report(true_classes, predicted_classes, target_names=class_labels)
        print("Classification Report:")
        print("=====================")
        print(report)

# Function to plot confusion matrix
def plot_confusion_matrix(model, generator, class_labels):
        # Predict the classes
        predictions = model.predict(generator)
        predicted_classes = np.argmax(predictions, axis=1)
        true_classes = generator.classes
        class_labels = list(generator.class_indices.keys())

        # Generate the confusion matrix
        cm = confusion_matrix(true_classes, predicted_classes)
        
        # Plot the confusion matrix
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
        disp.plot(cmap=plt.cm.Blues)
        plt.xticks(rotation=45)
        plt.show()
        
def create_transfer_learning_model(base_model_name, input_shape, num_classes):
        if base_model_name == 'VGG16':
                base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
        elif base_model_name == 'ResNet50':
                base_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
        else:
                raise ValueError("Base model not supported. Choose 'VGG16' or 'ResNet50'.")
        base_model.trainable = False
        model = Sequential([
                base_model,
                GlobalAveragePooling2D(),
                Dense(512, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(num_classes, activation='softmax')
        ])
        return model

EDA:

In [ ]:
# List the directories (each directory represents a class)
classes = os.listdir(data_dir)

# Print the classes
print(f"Classes: {classes}")

# Load a sample image from each class and display it
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.flatten()

for ax, class_name in zip(axes, classes):
    class_dir = os.path.join(data_dir, class_name)
    sample_image_path = os.path.join(class_dir, os.listdir(class_dir)[0])
    image = Image.open(sample_image_path)
    ax.imshow(image)
    ax.set_title(class_name)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Initialize a dictionary to store the count of images in each class
class_counts = {}

# Loop through each class directory and count the images
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    num_images = len(os.listdir(class_dir))
    class_counts[class_name] = num_images

# Convert the dictionary to a DataFrame for easy plotting
class_counts_df = pd.DataFrame(list(class_counts.items()), columns=['Class', 'Number of Images'])

# Plot the class distribution
plt.figure(figsize=(10, 6))
sns.barplot(x='Class', y='Number of Images', data=class_counts_df)
plt.title('Class Distribution')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Checking image dimensions
image_shapes = []

for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = Image.open(image_path)
        image_shapes.append(image.size)

# Convert to DataFrame for analysis
image_shapes_df = pd.DataFrame(image_shapes, columns=['Width', 'Height'])

# Plot image dimensions
plt.figure(figsize=(10, 6))
sns.histplot(data=image_shapes_df, x='Width', kde=True, color='blue', label='Width')
sns.histplot(data=image_shapes_df, x='Height', kde=True, color='red', label='Height')
plt.title('Image Dimensions Distribution')
plt.legend()
plt.show()


Data Preparation

In [ ]:
target_size = 264

train_data_generator = ImageDataGenerator(rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator = train_data_generator.flow_from_directory(
        train_dir,
        target_size=(target_size, target_size),
        batch_size=64,
        class_mode='categorical',
        shuffle=True)

val_data_generator = ImageDataGenerator(rescale=1./255)
val_generator = val_data_generator.flow_from_directory(
        val_dir,
        target_size=(target_size, target_size),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)

test_data_generator = ImageDataGenerator(rescale=1./255)
test_set = test_data_generator.flow_from_directory(
        test_dir,
        target_size=(target_size, target_size),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)

In [ ]:
# Model architecture
model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(target_size, target_size, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(512, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(1024, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
])

model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('other_test.h5', monitor='val_accuracy', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=4, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

print("Model initialized and Callbacks defined.")

In [ ]:
# Train the model with callbacks
history = model.fit(train_generator, epochs=50, validation_data=val_generator, 
                        callbacks=[early_stopping, model_checkpoint, reduce_lr])

In [ ]:
# Create a figure
plt.figure(figsize=(12, 5))

# Subplot for Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Subplot for Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()


Validation set:

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

In [ ]:
# Plot the confusion matrix
plot_confusion_matrix(model, val_generator, list(val_generator.class_indices.keys()))

In [ ]:
generate_classification_report(model, val_generator, list(val_generator.class_indices.keys()))

Testing Set:

In [ ]:
plot_confusion_matrix(model,test_set, list(test_set.class_indices.keys()))

In [ ]:
generate_classification_report(model, test_set, list(test_set.class_indices.keys()))

In [ ]:
display_images_with_labels(test_set, model, 31, 5, 6)

Test Other Models using Transfer Learning

input_shape = (target_size, target_size, 3)
num_classes = 10

vgg16_model = create_transfer_learning_model('VGG16', input_shape, num_classes)
vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

resnet50_model = create_transfer_learning_model('ResNet50', input_shape, num_classes)
resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Transfer Learning Models Created and Compiled.")

early_stopping = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=4, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

vgg16_history = vgg16_model.fit(train_generator, epochs=10, validation_data=val_generator, 
                                callbacks=[early_stopping, model_checkpoint, reduce_lr])

vgg16_loss, vgg16_accuracy = vgg16_model.evaluate(val_generator)
print("VGG16 Validation Loss:", vgg16_loss)
print("VGG16 Validation Accuracy:", vgg16_accuracy)

resnet50_history = resnet50_model.fit(train_generator, epochs=10, validation_data=val_generator, 
                                        callbacks=[early_stopping, model_checkpoint, reduce_lr])

resnet50_loss, resnet50_accuracy = resnet50_model.evaluate(val_generator)
print("ResNet50 Validation Loss:", resnet50_loss)
print("ResNet50 Validation Accuracy:", resnet50_accuracy)

def plot_training_history(history, title):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'{title} - Training and Validation Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title(f'{title} - Training and Validation Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_training_history(vgg16_history, 'VGG16')
plot_training_history(resnet50_history, 'ResNet50')

plot_confusion_matrix(vgg16_model, val_generator, list(val_generator.class_indices.keys()))
generate_classification_report(vgg16_model, val_generator, list(val_generator.class_indices.keys()))

plot_confusion_matrix(resnet50_model, val_generator, list(val_generator.class_indices.keys()))
generate_classification_report(resnet50_model, val_generator, list(val_generator.class_indices.keys()))

display_images_with_labels(test_set, vgg16_model, 31, 5, 6)
display_images_with_labels(test_set, resnet50_model, 31, 5, 6)

________________________________________________________________________________________________________________________________________
________________________________________________________________________________________________________________________________________
________________________________________________________________________________________________________________________________________
________________________________________________________________________________________________________________________________________
________________________________________________________________________________________________________________________________________


Loaded Model Procedure:

In [ ]:
# Load the saved model
loaded_model = load_model('last_test.h5')

print("Our best model:")
loaded_model.summary()

In [ ]:
# Evaluate the model
loaded_loss, loaded_accuracy = loaded_model.evaluate(val_generator)
print("Evaluation Results for the Loaded Model:")
print("=========================================")
print(f"Validation Loss: {loaded_loss}")
print(f"Validation Accuracy: {loaded_accuracy}")

In [ ]:
plot_confusion_matrix(loaded_model, val_generator, list(val_generator.class_indices.keys()))

In [ ]:
generate_classification_report(loaded_model, val_generator, list(val_generator.class_indices.keys()))

In [ ]:
plot_confusion_matrix(loaded_model, test_set, list(test_set.class_indices.keys()))

In [ ]:
generate_classification_report(loaded_model, test_set, list(test_set.class_indices.keys()))

In [ ]:
display_images_with_labels(test_set, loaded_model, 30, 5, 6)